In [155]:
import numpy as np
import pandas as pd
from python_module.pricing_model import BSMModel, SABRModel
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import plotly.express as px

In [156]:
# Inputs
F = 100
T = 1
alpha = 0.2
beta = 1
rho = -0.5
nu = 0.9
r = 0

In [157]:
# Generate options
results = dict()
for K in range(60, 141, 1):
    option_type = 'call' if K > F else 'put'
    out = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type=option_type)
    out['option_type'] = option_type
    out['delta_call'] = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type='call')['delta']
    results[K] = out

# Compute COGWA
df = pd.DataFrame(results)
df = df.transpose()
X = df[['gamma', 'vanna', 'volga']]
y = df['theta']
model = LinearRegression(fit_intercept=False)
model.fit(X.values, y.values)
y_pred = model.predict(X.values)
coef = model.coef_
r2 = r2_score(y.values, y_pred)
cogwa = (X * (coef))
cogwa.columns = ['costof_gamma', 'costof_vanna', 'costof_volga']
df = pd.concat([df, cogwa], axis=1)

# Generate Weights
df['weight'] = (1 / np.power(df.index, 2)) * 1_000_000

# Delta Bucketing
bins = [0, 0.25, 0.5, 0.75, 1]
labels = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1']
df['delta_bucket'] = pd.cut(df['delta_call'], bins=bins, labels=labels)

# Compute risk adjusted to quantity
risk_ref = 'theta'
df[f'pos_{risk_ref}'] = df[risk_ref] * df['weight']

# Compute quantity reduction with discretionay approch
df['risk_bucket'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)[f'pos_{risk_ref}'].sum().to_dict()))
df['qty_to_trade'] = df['risk_bucket'] / df[risk_ref]
df['min_qty_to_trade'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)['qty_to_trade'].max().to_dict())) 
df['weight_reduction'] = (df['qty_to_trade']==df['min_qty_to_trade']) * df['min_qty_to_trade']
df['weight_reduction'] = df['weight_reduction'].replace(0, np.nan)

# Compute quantity reduction with lasso
X = df[['costof_gamma', 'costof_vanna', 'costof_volga']].transpose()
y = df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight'], axis=0).sum()
max_iteration = 100
alpha = 0.01
lasso = Lasso(alpha=alpha)
lasso.fit(X.values, y)
df['lasso'] = lasso.coef_
df['lasso'] = df['lasso'].replace(0, np.nan)

# Compute quantity reduction with matrix inversion approach
# Compute coefficients for X and y (align indices)
X_sub = df.loc[df['weight_reduction'].dropna().index, ['costof_gamma','costof_vanna','costof_volga']].transpose()
y_sub = y.loc[X_sub.index] 
y_sub = y.loc[X_sub.index]     # make sure y aligns with X
X_sub = X_sub.iloc[:, 1:]
X_sub = X_sub.astype(float)
y_sub = y_sub.astype(float)
weight = np.linalg.solve(X_sub, y_sub)
df.loc[X_sub.columns, 'weight_matrix_inv'] = weight

In [182]:
test = df[['costof_gamma','costof_vanna','costof_volga']].multiply(df['weight'], axis=0)
costof_gamma_strike = test['costof_gamma'].abs().idxmax()
costof_vanna_strike = test['costof_vanna'].abs().idxmax()
costof_volga_strike = test['costof_volga'].abs().idxmax()
X_sub = df.loc[[costof_gamma_strike, costof_vanna_strike, costof_volga_strike], ['costof_gamma','costof_vanna','costof_volga']].transpose()
y_sub = y.loc[X_sub.index] 
X_sub = X_sub.astype(float)
y_sub = y_sub.astype(float)
weight = np.linalg.solve(X_sub, y_sub)
df.loc[X_sub.columns, 'weight_matrix_inv'] = weight

In [183]:
index_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight'], axis=0).sum()
disc_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight_reduction'], axis=0).sum()
lasso_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['lasso'], axis=0).sum()
matrix_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight_matrix_inv'], axis=0).sum()

In [184]:
lasso_cog-index_cog

costof_gamma    19.102824
costof_vanna    16.750515
costof_volga    16.933438
dtype: object

In [185]:
df['theta'].dot(df['lasso'].fillna(0))  - df['pos_theta'].sum()

52.92069180206754

In [186]:
px.scatter(df[['weight', 'weight_matrix_inv', 'weight_reduction', 'lasso']])

In [162]:
df['x'] = df['weight_reduction']*df['theta']

In [163]:
df['x'].sum()

-110.7434704054444

In [164]:
df['pos_theta'].sum()

-110.74347040544448

In [165]:
df.groupby('delta_bucket', observed=False)[['pos_theta', 'x']].sum()

,pos_theta,x
delta_bucket,,
0-0.25,-11.694179,-11.694179
0.25-0.5,-15.2733,-15.2733
0.5-0.75,-26.563704,-26.563704
0.75-1,-57.212287,-57.212287


In [166]:
df

,IV,price,delta,gamma,vega,theta,vanna,volga,option_type,delta_call,...,weight,delta_bucket,pos_theta,risk_bucket,qty_to_trade,min_qty_to_trade,weight_reduction,lasso,weight_matrix_inv,x
60,0.349856,0.859806,-0.051021,0.002996,0.10481,-0.007276,-0.385015,62.951159,put,0.948979,...,277.777778,0.75-1,-2.020975,-57.212287,7863.682227,7863.682227,7863.682227,NaN,NaN,-57.212287
61,0.345156,0.910939,-0.054283,0.00319,0.110093,-0.00754,-0.401744,64.466886,put,0.945717,...,268.744961,0.75-1,-2.026214,-57.212287,7588.2979,7863.682227,NaN,NaN,NaN,NaN
62,0.340515,0.964874,-0.05773,0.003394,0.115572,-0.007808,-0.418689,65.906473,put,0.94227,...,260.145682,0.75-1,-2.031309,-57.212287,7327.064435,7863.682227,NaN,NaN,NaN,NaN
63,0.335933,1.021766,-0.061374,0.003609,0.121251,-0.008082,-0.435799,67.258812,put,0.938626,...,251.952633,0.75-1,-2.036225,-57.212287,7079.170862,7863.682227,NaN,NaN,NaN,NaN
64,0.331408,1.081784,-0.065223,0.003836,0.127132,-0.00836,-0.45302,68.512211,put,0.934777,...,244.140625,0.75-1,-2.040928,-57.212287,6843.868382,7863.682227,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0.19528,0.559886,0.069846,0.006864,0.13404,-0.005194,1.147811,169.524823,call,0.069846,...,54.065744,0-0.25,-0.280792,-11.694179,2251.679748,2600.282424,NaN,NaN,NaN,NaN
137,0.196347,0.528069,0.066141,0.006545,0.128516,-0.005007,1.113701,167.630593,call,0.066141,...,53.279344,0-0.25,-0.266753,-11.694179,2335.710937,2600.282424,NaN,NaN,NaN,NaN
138,0.197433,0.498642,0.062683,0.006243,0.123264,-0.004829,1.080139,165.546173,call,0.062683,...,52.509977,0-0.25,-0.253552,-11.694179,2421.834132,2600.282424,NaN,NaN,NaN,NaN
139,0.198537,0.471392,0.059453,0.005957,0.118271,-0.004659,1.047216,163.300972,call,0.059453,...,51.757155,0-0.25,-0.241135,-11.694179,2510.030712,2600.282424,NaN,NaN,NaN,NaN
